In [ ]:
### nohup python3 -u TRAINevaluate_proteins0525validsmalltest.py > TRAINevaluate_proteins0525validsmalltestOUT.txt 2>&1 &

import re
from re import search
import os, sys
from os import walk
import traceback
import subprocess
import pandas as pd
import time
import math
import numpy as np

import sklearn
from sklearn.model_selection import train_test_split
from autogluon.tabular import FeatureMetadata
from autogluon.tabular import TabularDataset, TabularPredictor

def column_del(name_columnD,df_temp_del, column_drop_list):
    for i, columndel in enumerate (column_drop_list):
        try:
            df_temp_del=df_temp_del.drop(columns=[columndel], axis=1)
            #print ('Delete column : ', columndel ,'   -   ', name_columnD)
        except:     # KeyError
            #print ('Cannot find the column name : ', columndel,'   -   ', name_columnD)
            print (' ')
    return (df_temp_del)

def dataset_split_trainvalidtest (dataload, trainfilein):
    trainprotein=re.sub('\.csv','',trainfilein)
    traindata_split_filename=trainprotein+'_train.csv'
    validdata_split_filename=trainprotein+'_valid.csv'
    trainsize=0.8 # 0.85 or 0.7
    traindata, validdata = train_test_split(dataload, train_size=trainsize) #random_state=777,
    os.chdir(workpath)
    traindata.to_csv(traindata_split_filename,index=False,encoding='utf-8')
    validdata.to_csv(validdata_split_filename,index=False,encoding='utf-8')
    print ('Split dataset : ', trainfilein, ' -> ', traindata_split_filename,' & ', validdata_split_filename)
    return (traindata, validdata)


def AUTOGLUON_predect(workpath, ML_path, predfilein):
    os.chdir(workpath)
    predfileout=re.sub('.csv','',predfilein)+'_predictout.csv'
    preddata=TabularDataset(predfilein)
    predictor = TabularPredictor.load(ML_path)      # to load previously-trained predictor from file
    preddata_label = predictor.predict(preddata)
    print("Predictions:  \n", preddata_label)
    preddata_and_predLABEL=pd.concat([preddata,preddata_label], axis=1)
    preddata_and_predLABEL.to_csv(predfileout,index=False,encoding='utf-8')
    print('\n', time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n')
    return(predfileout, preddata_and_predLABEL)


def TRAINevaluateDATA(workpath,predictor,ML_path,fileevaluation,AG_ATG,AIlabel_problem_eval,df_TRAINevaluate,trainnum):  
    os.chdir(workpath)
    print ('=== ', fileevaluation, ' ===')
    # AIlabel_problem_eval=[ec50_label,problem_type,eval_metric,AgonistAnagonist_label]
    AIlabel=AIlabel_problem_eval[0]
    AgonistAnagonist_label=AIlabel_problem_eval[3]

    
    try:
        #predictor = TabularPredictor.load(ML_path)
        test_data0=TabularDataset(fileevaluation)  #validdata

        ML_path_split=re.split('\/',ML_path)
        ML_path_AgonistAntagonist_test=ML_path_split[-1]
        
        

#        if re.search('_AgonistAntagonist',ML_path_AgonistAntagonist_test):
#            test_data=test_data0.copy()
#            AgonistAntagonist_test='AgonistAntagonist'
#        elif re.search('_Agonist',ML_path_AgonistAntagonist_test):
#            test_data=test_data0.loc[test_data0[AgonistAnagonist_label]==1]
#            AgonistAntagonist_test='Agonist'
#        elif re.search('_Antagonist',ML_path_AgonistAntagonist_test):
#            test_data=test_data0.loc[test_data0[AgonistAnagonist_label]==-1]
#            AgonistAntagonist_test='Antagonist'
#        elif re.search('_Anagonist',ML_path_AgonistAntagonist_test):
#            test_data=test_data0.loc[test_data0[AgonistAnagonist_label]==-1]
#            AgonistAntagonist_test='Antagonist'
#        else:
#            AgonistAntagonist_test='AgonistAntagonist_check'
#            print('*** check ML_path  ***', ML_path_AgonistAntagonist_test,AgonistAntagonist_test)

        if AG_ATG=='AG':
            test_data=test_data0.loc[test_data0[AgonistAnagonist_label]==1]
            AgonistAntagonist_test='Agonist'
        elif AG_ATG=='ATG':
            test_data=test_data0.loc[test_data0[AgonistAnagonist_label]==-1]
            AgonistAntagonist_test='Antagonist'
        else:
            test_data=test_data0.copy()
            AgonistAntagonist_test='AgonistAntagonist'


        if re.search('HUMAN',ML_path_AgonistAntagonist_test):
            species='HUMAN'
        elif re.search('HUMNA',ML_path_AgonistAntagonist_test):
            species='HUMAN'
        elif re.search('human',ML_path_AgonistAntagonist_test):
            species='HUMAN'
        elif re.search('INTER',ML_path_AgonistAntagonist_test):
            species='INTERSPECIES'
        elif re.search('all',ML_path_AgonistAntagonist_test):
            species='ALLCONCAT'
        else:
            species='INTERSPECIES'
            print('*** check ML_path  ***', ML_path_AgonistAntagonist_test,species)


        ### 
        #test_data=test_data.dropna() ## copy()  ## clean data for na  # be careful : 'EfficacyValue_percentage'
        test_data=test_data.reset_index(drop=True)

        df_TRAINevaluate.at[trainnum,'SPECIES']=species
        df_TRAINevaluate.at[trainnum,'AgonistAntagonist_test']=AgonistAntagonist_test
        df_TRAINevaluate.at[trainnum,'test_data size']="{:.0f}".format(len(test_data))
        
        try:
            test_evaluate=predictor.evaluate(test_data)
            print (' \n ++++++++++ \n ', test_evaluate,' \n ++++++++++ \n')
            df_TRAINevaluate.at[trainnum,'mean_squared_error']=abs(test_evaluate['mean_squared_error'])
            df_TRAINevaluate.at[trainnum,'pearsonr']=test_evaluate['pearsonr']
            df_TRAINevaluate.at[trainnum,'mean_absolute_error']=abs(test_evaluate['mean_absolute_error'])
            df_TRAINevaluate.at[trainnum,'root_mean_squared_error']=abs(test_evaluate['root_mean_squared_error'])
            df_TRAINevaluate.at[trainnum,'r2']="{:.5f}".format(abs(test_evaluate['r2']))
            df_TRAINevaluate.at[trainnum,'median_absolute_error']=abs(test_evaluate['median_absolute_error'])
        except:
            print ('No test evaluate')
    except:
        print ('*** wrong data ***')
    return (df_TRAINevaluate,trainnum)



def proteinfiles_evaluation_test(workpath,predictor,ML_path,m,proteinpath,proteinfiles,evaluation_type,AG_ATG,AIlabel_problem_eval,df_TRAINevaluate,trainnum,outputfile,index_vectors_len):
    workpath_home='/home1/drhuangwc/AAALAB/GPCR5b/Interspecies_20220915_AgonistAntagonist_datacode2ALLcolumns/seperatePROTEIN'

    starttime1=time.time()
    peotein_index_len,padel_index_len,ECFP_index_len=index_vectors_len
    for i,ppp_evaluate in enumerate(proteinfiles):
        print ('\n\n ----------- \n',m, ML_path,'\n', i, ppp_evaluate, '\n ----------- ')
        fileevaluation=proteinpath+'/'+ppp_evaluate
        df_TRAINevaluate.at[trainnum,'ML_fullpath']=ML_path
        df_TRAINevaluate.at[trainnum,'peotein_index']="{:.0f}".format(peotein_index_len)
        df_TRAINevaluate.at[trainnum,'padel_index']="{:.0f}".format(padel_index_len)
        df_TRAINevaluate.at[trainnum,'ECFP_index']="{:.0f}".format(ECFP_index_len)
        df_TRAINevaluate.at[trainnum,'file_evaluate']=ppp_evaluate
        proteinname=re.sub('GPCR_to_trainAgonistAntagonist_','',ppp_evaluate)
        proteinname=re.sub('GPCR_to_trainAgonist_','',proteinname)
        proteinname=re.sub('GPCR_to_trainAntagonist_','',proteinname)
        proteinname=re.sub('GPCR_to_test_','',proteinname)
        proteinname=re.sub('_valid.csv','',proteinname)
        proteinname=re.sub('_train.csv','',proteinname)
        proteinname=re.sub('_smalldataset.csv','',proteinname)
        df_TRAINevaluate.at[trainnum,'Name']=proteinname
        df_TRAINevaluate.at[trainnum,'evaluate']=evaluation_type
        df_TRAINevaluate.at[trainnum,'AG_ATG']=AG_ATG
        df_TRAINevaluate,trainnum=TRAINevaluateDATA(workpath,predictor,ML_path,fileevaluation,AG_ATG,AIlabel_problem_eval,df_TRAINevaluate,trainnum)
        trainnum=trainnum+1
        os.chdir(workpath_home)
        df_TRAINevaluate.to_csv(outputfile,index=False,encoding='utf-8')
    print(time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n sub-total = ', (time.time()-starttime1)/3600, ' hrs \n +++++++++++++ \n'  )
    return(df_TRAINevaluate,trainnum)

######################################## Main ########################################
timestamp=time.strftime("%Y%m%d%H%M",time.localtime(time.time()))
starttime=time.time()


starttime1=time.time()

ec50_label='ActivityValue_log_EC50'
AgonistAnagonist_label='AgonistP1AnagonistN1_label'
problem_type ='regression'  
eval_metric='mean_squared_error'

AIlabel_problem_eval=[ec50_label,problem_type,eval_metric,AgonistAnagonist_label]
substr=['GPCR_to_train','_train.csv','_valid.csv']

df_TRAINevaluate=pd.DataFrame()
trainnum=0
outputfile='TRAINevaluate_proteins_valid_small_test_'+timestamp+'.csv'

homepath=str(os.getcwd())
workpath_home=homepath+'/Human_AgonistAntagonist_dataprocess/GPCR_separatePROTEIN'
workpath_dirs=['AgonistAntagonist','Agonist','Antagonist']


for workpath_dir in workpath_dirs:
    workpath=workpath_home+'/'+workpath_dir
    proteinpath=workpath
    
    os.chdir(workpath)
    paths = next(walk(workpath), (None, None, []))[1]
    MLpaths=[]
    for path in paths:
        if re.search('AutogluonTRAIN',path):
            MLpaths.append(path)

    ###########################

    files=next(walk(workpath),(None,None,[]))[2]
    humanfilesvalidAGATG=[]
    humanfilesvalidAG=[]
    humanfilesvalidATG=[]
    humanfilesmall=[]
    mrmrfilestrain=[]
    mrmrfilesvalid=[]
    for file in files:
        if re.search('_HUMAN_valid.csv',file):
            if re.search('_trainAgonistAntagonist_',file):
                humanfilesvalidAGATG.append(file) 
            if re.search('_trainAgonist_',file):
                humanfilesvalidAG.append(file)
            elif re.search('_trainAntagonist_',file):
                humanfilesvalidATG.append(file)

        if re.search('_HUMAN_smalldataset.csv',file):
            humanfilesmall.append(file)
    humanfilesvalidAGATG0=sorted(humanfilesvalidAGATG)
    humanfilesvalidAG0=sorted(humanfilesvalidAG)
    humanfilesvalidATG0=sorted(humanfilesvalidATG)
    humanfilesmall0=sorted(humanfilesmall)
    print ('ML_paths: ',MLpaths,len(MLpaths))
    print ('humanfilesvalidAGATG files: ',humanfilesvalidAGATG0,len(humanfilesvalidAGATG0))
    print ('humanfilesvalidAG files: ',humanfilesvalidAG0,len(humanfilesvalidAG0))
    print ('humanfilesvalidATG files: ',humanfilesvalidATG0,len(humanfilesvalidATG0))
    print ('humanfilesmall files: ',humanfilesmall0,len(humanfilesmall0))

    for m, ML_path in enumerate(MLpaths):

        ML_path_split=re.split('_',ML_path)

        try:
            if re.search('_HUMAN_',ML_path):
                trainfile='GPCR_to_train'+ML_path_split[1]+'_'+ML_path_split[2]+'_'+ML_path_split[3]+'_train.csv'
                validfile='GPCR_to_train'+ML_path_split[1]+'_'+ML_path_split[2]+'_'+ML_path_split[3]+'_valid.csv'

                if 'AgonistAntagonist' in ML_path_split:
                    humanfilesvalidAGATG=[validfile]
                    humanfilesvalidAG=[]
                    humanfilesvalidATG=[]
                elif 'Agonist' in ML_path_split:
                    humanfilesvalidAGATG=[]
                    humanfilesvalidAG=[validfile]
                    humanfilesvalidATG=[]
                elif 'Antagonist' in ML_path_split:
                    humanfilesvalidAGATG=[]
                    humanfilesvalidAG=[]
                    humanfilesvalidATG=[validfile]
                else:
                    print ('please check: ',ML_path_split)
                    humanfilesvalidAGATG=humanfilesvalidAGATG0.copy()
                    humanfilesvalidAG=humanfilesvalidAG0.copy()
                    humanfilesvalidATG=humanfilesvalidATG0.copy()
            else:
                trainfile='GPCR_to_train'+ML_path_split[1]+'_'+ML_path_split[2]+'_train.csv'
                validfile='GPCR_to_train'+ML_path_split[1]+'_'+ML_path_split[2]+'_valid.csv'

                humanfilesvalidAGATG=humanfilesvalidAGATG0.copy()
                humanfilesvalidAG=humanfilesvalidAG0.copy()
                humanfilesvalidATG=humanfilesvalidATG0.copy()
        except:
            
            humanfilesvalidAGATG=humanfilesvalidAGATG0.copy()
            humanfilesvalidAG=humanfilesvalidAG0.copy()
            humanfilesvalidATG=humanfilesvalidATG0.copy()

        try:
            os.chdir(workpath)
            trainfileload=pd.read_csv(trainfile)
            peotein_index= [p for p in trainfileload.columns if 'Protein_code1' in p]
            padel_index= [p for p in trainfileload.columns if 'padel1d2d' in p]
            ECFP_index= [p for p in trainfileload.columns if 'ECFP' in p]
            index_vectors_len=[len(peotein_index),len(padel_index),len(ECFP_index)]    
            
        except:
            print ('can not load trainfile',trainfile)
            index_vectors_len=[2554,1444,1024]
            
        print ('index_vectors_len: ', index_vectors_len)

        os.chdir(workpath)
        predictor = TabularPredictor.load(ML_path)

        if len(humanfilesvalidAGATG)>0:
            df_TRAINevaluate,trainnum=proteinfiles_evaluation_test(workpath=workpath,predictor=predictor,ML_path=ML_path,m=m,proteinpath=proteinpath,proteinfiles=humanfilesvalidAGATG,evaluation_type='human_valid',AG_ATG='AG',AIlabel_problem_eval=AIlabel_problem_eval,df_TRAINevaluate=df_TRAINevaluate,trainnum=trainnum,outputfile=outputfile,index_vectors_len=index_vectors_len)
            df_TRAINevaluate,trainnum=proteinfiles_evaluation_test(workpath=workpath,predictor=predictor,ML_path=ML_path,m=m,proteinpath=proteinpath,proteinfiles=humanfilesvalidAGATG,evaluation_type='human_valid',AG_ATG='ATG',AIlabel_problem_eval=AIlabel_problem_eval,df_TRAINevaluate=df_TRAINevaluate,trainnum=trainnum,outputfile=outputfile,index_vectors_len=index_vectors_len)
            df_TRAINevaluate,trainnum=proteinfiles_evaluation_test(workpath=workpath,predictor=predictor,ML_path=ML_path,m=m,proteinpath=proteinpath,proteinfiles=humanfilesvalidAGATG,evaluation_type='human_valid',AG_ATG='AGATG',AIlabel_problem_eval=AIlabel_problem_eval,df_TRAINevaluate=df_TRAINevaluate,trainnum=trainnum,outputfile=outputfile,index_vectors_len=index_vectors_len)
        if len(humanfilesvalidAG)>0:
            df_TRAINevaluate,trainnum=proteinfiles_evaluation_test(workpath=workpath,predictor=predictor,ML_path=ML_path,m=m,proteinpath=proteinpath,proteinfiles=humanfilesvalidAG,evaluation_type='human_valid',AG_ATG='AG',AIlabel_problem_eval=AIlabel_problem_eval,df_TRAINevaluate=df_TRAINevaluate,trainnum=trainnum,outputfile=outputfile,index_vectors_len=index_vectors_len)
        if len(humanfilesvalidATG)>0:
            df_TRAINevaluate,trainnum=proteinfiles_evaluation_test(workpath=workpath,predictor=predictor,ML_path=ML_path,m=m,proteinpath=proteinpath,proteinfiles=humanfilesvalidATG,evaluation_type='human_valid',AG_ATG='ATG',AIlabel_problem_eval=AIlabel_problem_eval,df_TRAINevaluate=df_TRAINevaluate,trainnum=trainnum,outputfile=outputfile,index_vectors_len=index_vectors_len)
        if len(humanfilesmall)>0:
            df_TRAINevaluate,trainnum=proteinfiles_evaluation_test(workpath=workpath,predictor=predictor,ML_path=ML_path,m=m,proteinpath=proteinpath,proteinfiles=humanfilesmall,evaluation_type='human_small',AG_ATG='AGATG',AIlabel_problem_eval=AIlabel_problem_eval,df_TRAINevaluate=df_TRAINevaluate,trainnum=trainnum,outputfile=outputfile,index_vectors_len=index_vectors_len)


    print(time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n sub-total = ', (time.time()-starttime1)/3600, ' hrs \n'  )


###

os.chdir(workpath_home)
df_TRAINevaluate.to_csv(outputfile,index=False,encoding='utf-8')

if (time.time()-starttime)/3600 > 1 :
    print(time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n Total = ', (time.time()-starttime)/3600, ' hrs \n' )
else:
    print(time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n Total = ', (time.time()-starttime)/60, ' mins \n' )



In [4]:
import os
import pandas as pd
homepath=str(os.getcwd())
workpath_home=homepath+'/Human_AgonistAntagonist_dataprocess/GPCR_separatePROTEIN'
os.chdir(workpath_home)
df_TRAINevaluate=pd.read_csv('TRAINevaluate_proteins_valid_small_test_all_AG_ATG_AGATG_202305251611.csv')
df_TRAINevaluate

,ML_fullpath,peotein_index,padel_index,ECFP_index,file_evaluate,Name,evaluate,AG_ATG,SPECIES,AgonistAntagonist_test,test_data size,mean_squared_error,pearsonr,mean_absolute_error,root_mean_squared_error,r2,median_absolute_error
0,AutogluonTRAIN0519_AgonistAntagonist_PD2R2_HUM...,2554,1444,1024,GPCR_to_trainAgonistAntagonist_PD2R2_HUMAN_val...,PD2R2_HUMAN,human_valid,AG,HUMAN,Agonist,0,NaN,NaN,NaN,NaN,NaN,NaN
1,AutogluonTRAIN0519_AgonistAntagonist_PD2R2_HUM...,2554,1444,1024,GPCR_to_trainAgonistAntagonist_PD2R2_HUMAN_val...,PD2R2_HUMAN,human_valid,ATG,HUMAN,Antagonist,336,0.146132,0.925786,0.252535,0.382272,0.85597,0.153836
2,AutogluonTRAIN0519_AgonistAntagonist_PD2R2_HUM...,2554,1444,1024,GPCR_to_trainAgonistAntagonist_PD2R2_HUMAN_val...,PD2R2_HUMAN,human_valid,AGATG,HUMAN,AgonistAntagonist,338,0.154547,0.922162,0.258451,0.393125,0.84927,0.154135
3,AutogluonTRAIN0519_AgonistAntagonist_PD2R2_HUM...,2554,1444,1024,GPCR_to_trainAgonistAntagonist_OXGR1_HUMAN_sma...,OXGR1_HUMAN,human_small,AGATG,HUMAN,AgonistAntagonist,3,1.454604,-0.395881,1.177639,1.206070,26.51922,1.347330
4,AutogluonTRAIN0519_AgonistAntagonist_PD2R2_HUM...,2554,1444,1024,GPCR_to_trainAgonistAntagonist_CXCR1_HUMAN_sma...,CXCR1_HUMAN,human_small,AGATG,HUMAN,AgonistAntagonist,1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13617,AutogluonTRAIN0518_Antagonist_CCR6_HUMAN_2min,2554,1444,1024,GPCR_to_trainAntagonist_PF2R_HUMAN_smalldatase...,PF2R_HUMAN,human_small,AGATG,HUMAN,AgonistAntagonist,3,0.051550,0.689050,0.215201,0.227047,4.30415,0.224266
13618,AutogluonTRAIN0518_Antagonist_CCR6_HUMAN_2min,2554,1444,1024,GPCR_to_trainAntagonist_P2Y11_HUMAN_smalldatas...,P2Y11_HUMAN,human_small,AGATG,HUMAN,AgonistAntagonist,2,0.170927,NaN,0.413434,0.413434,0.00000,0.413434
13619,AutogluonTRAIN0518_Antagonist_CCR6_HUMAN_2min,2554,1444,1024,GPCR_to_trainAntagonist_T2R31_HUMAN_smalldatas...,T2R31_HUMAN,human_small,AGATG,HUMAN,AgonistAntagonist,2,0.307254,NaN,0.554305,0.554305,0.00000,0.554305
13620,AutogluonTRAIN0518_Antagonist_CCR6_HUMAN_2min,2554,1444,1024,GPCR_to_trainAntagonist_HRH2_HUMAN_smalldatase...,HRH2_HUMAN,human_small,AGATG,HUMAN,AgonistAntagonist,7,0.983250,-0.939944,0.873871,0.991590,4.51260,0.838996
